In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv('/content/drive/MyDrive/movies.csv', sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [10]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
ratings.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


### Content Based

**Objective**: Build a Content-Based Recommendation system that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

**Dataset**:
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Split the `genres` column into separate genre strings.
   - Fill any missing values in the `genres` column and convert it to string.

2. **Compute TF-IDF Matrix**:
   - Use `TfidfVectorizer` to transform the genres into a TF-IDF matrix.

3. **Calculate Cosine Similarity**:
   - Compute cosine similarity between the TF-IDF matrices of movies.

4. **Recommend Movies**:
   - Create a function that gets movie recommendations based on the cosine similarity score of movie genres.

In [12]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [14]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [15]:
def genre_recommendations(title, n=10):
    idx = movies.loc[movies['title'].isin([title])].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(n+1)]
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]

In [16]:
# Example
genre_recommendations('GoldenEye (1995)')

345     Clear and Present Danger (1994)
543           Surviving the Game (1994)
724                    Rock, The (1996)
788                     Daylight (1996)
825               Chain Reaction (1996)
978                 Maximum Risk (1996)
1467                    Anaconda (1997)
1513                     Con Air (1997)
1693                   Firestorm (1998)
3686          Perfect Storm, The (2000)
Name: title, dtype: object


### Collaborative Filtering

**Objective**: Implement a collaborative filtering movie recommendation system to recommend movies to a user based on similar users' spreferences.

**Dataset**:

- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Split `genres` column into separate rows.

2. **Create User-Genre Matrix**:
   - Create a matrix where rows are users and columns are genres.
   - Each cell represents the ratio of movies watched by the user in that genre to the total movies watched by the user.

3. **Calculate User Similarity**:
   - Use cosine similarity to calculate the similarity between users.

4. **Find Top N Similar Users**:
   - Find the top N users with the most similar preferences to the given user.

5. **Recommend Movies**:
   - Recommend movies that similar users have watched but the target user has not.
   - Return titles and genres of the top 10 recommended movies.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data = pd.merge(ratings, movies, on='movie_id')

In [ ]:
data.head(-11)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...
1000193,5227,591,3,5226,590,Tough and Deadly (1995),Action|Drama|Thriller
1000194,5795,591,1,5794,590,Tough and Deadly (1995),Action|Drama|Thriller
1000195,5313,3656,5,5312,3655,Lured (1947),Crime
1000196,5328,2438,4,5327,2437,Outside Ozona (1998),Drama|Thriller


In [ ]:
data = data.assign(genres=data['genres'].str.split('|')).explode('genres')

In [ ]:
data.head(-11)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...
1000198,5334,127,1,5333,126,"Silence of the Palace, The (Saimt el Qusur) (1...",Drama
1000199,5334,3382,5,5333,3381,Song of Freedom (1936),Drama
1000200,5420,1843,3,5419,1842,Slappy and the Stinkers (1998),Children's
1000200,5420,1843,3,5419,1842,Slappy and the Stinkers (1998),Comedy


In [ ]:
def get_top_n_similar_users(user_id, n=5):
    user_profiles = data.groupby('user_id').apply(lambda x: pd.get_dummies(x['genres']).sum() / x['genres'].count())
    user_profiles = user_profiles.fillna(0)
    unique_genres = np.unique(data['genres'])
    genre2idx = {genre: i for i, genre in enumerate(unique_genres)}
    matrix = np.zeros((len(user_profiles), len(unique_genres)))
    for uu, value in user_profiles.items():
      id, genre = uu
      matrix[id-1, genre2idx[genre]] = value
    user = matrix[user_id-1]
    similarity = cosine_similarity(matrix, [user])
    similar_users = np.argsort(similarity[:,0])[-n-1:-1]
    return similar_users,similarity[similar_users]


In [ ]:
def recommend_movies(user_id, n_similar_users=5, n_recommendations=10):
    similar_users,similarity = get_top_n_similar_users(user_id, n_similar_users)
    user_data = data[data['user_id'] == user_id]
    user_movies = set(user_data['title'])
    recommendations = []
    movies_counter = {}
    for i in range(len(similar_users)):
        similar_user_data = data[data['user_id'] == similar_users[i]+1]
        similar_user_movies = set(similar_user_data['title'])
        for movie in similar_user_movies:
            if movie not in user_movies:
                if movie not in movies_counter:
                    movies_counter[movie] = 0
                movies_counter[movie] += similarity[i][0]
    sorted_movies = sorted(movies_counter.items(), key=lambda x: x[1], reverse=True)
    for movie, _ in sorted_movies[:n_recommendations]:
        recommendations.append(movie)
    recommendations = pd.merge(movies, pd.DataFrame(recommendations, columns=['title']), on='title')
    return recommendations.head(n_recommendations)


In [ ]:
# Example
user_id = 1
recommended_movies = recommend_movies(user_id, n_similar_users=5, n_recommendations=10)

In [ ]:
recommended_movies

,movie_id,title,genres
0,34,Babe (1995),Children's|Comedy|Drama
1,912,Casablanca (1942),Drama|Romance|War
2,1032,Alice in Wonderland (1951),Animation|Children's|Musical
3,1617,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
4,2078,"Jungle Book, The (1967)",Animation|Children's|Comedy|Musical
5,2080,Lady and the Tramp (1955),Animation|Children's|Comedy|Musical|Romance
6,2081,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
7,2087,Peter Pan (1953),Animation|Children's|Fantasy|Musical
8,2096,Sleeping Beauty (1959),Animation|Children's|Musical
9,2858,American Beauty (1999),Comedy|Drama


### SVD (Singular Value Decomposition)



**Objective**: Implement an SVD-based recommendation system to recommend movies to users by decomposing the user-item interaction matrix into latent factors.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Create a user-item matrix where rows represent users, columns represent movies, and the values are the ratings.

2. **Decompose Matrix using SVD**:
   - Apply Singular Value Decomposition (SVD) to decompose the user-item matrix into three matrices: $U$, $\Sigma$, and $V^T$.

3. **Reconstruct Matrix**:
   - Reconstruct the user-item matrix using the top $k$ singular values to reduce dimensionality.

4. **Predict Ratings**:
   - Use the reconstructed matrix to predict ratings for all user-item pairs.

5. **Recommend Movies**:
   - Recommend the top 20 movies with the highest predicted ratings for a given user that the user hasn't rated yet.

In [ ]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


Fill na elements.

In [ ]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import numpy as np

# calculate rating matrix
R = Ratings.to_numpy()
#print(R)
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)


In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

sigma = np.diag(sigma)


In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [ ]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.288861,0.143055,-0.195080,-0.018843,0.012232,-0.176604,-0.074120,0.141358,-0.059553,-0.195950,...,0.027807,0.001640,0.026395,-0.022024,-0.085415,0.403529,0.105579,0.031912,0.050450,0.088910
1,0.744716,0.169659,0.335418,0.000758,0.022475,1.353050,0.051426,0.071258,0.161601,1.567246,...,-0.056502,-0.013733,-0.010580,0.062576,-0.016248,0.155790,-0.418737,-0.101102,-0.054098,-0.140188
2,1.818824,0.456136,0.090978,-0.043037,-0.025694,-0.158617,-0.131778,0.098977,0.030551,0.735470,...,0.040481,-0.005301,0.012832,0.029349,0.020866,0.121532,0.076205,0.012345,0.015148,-0.109956
3,0.408057,-0.072960,0.039642,0.089363,0.041950,0.237753,-0.049426,0.009467,0.045469,-0.111370,...,0.008571,-0.005425,-0.008500,-0.003417,-0.083982,0.094512,0.057557,-0.026050,0.014841,-0.034224
4,1.574272,0.021239,-0.051300,0.246884,-0.032406,1.552281,-0.199630,-0.014920,-0.060498,0.450512,...,0.110151,0.046010,0.006934,-0.015940,-0.050080,-0.052539,0.507189,0.033830,0.125706,0.199244


In [ ]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):

    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = predictions.iloc[user_row_number].sort_values(ascending=False)

    user_data = original_ratings[original_ratings.user_id == (userID)]


    # merge movies data with user_data
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )


    print(f'User {userID} has already rated {user_full.shape[0]} movies.')
    print(f'Recommending highest {num_recommendations} predicted ratings movies not already rated.')

    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [ ]:
already_rated, predictions = recommend_movies(preds, 4375, movies, ratings, 20)

User 4375 has already rated 325 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [ ]:
predictions.head(20)

,movie_id,title,genres
470,539,Sleepless in Seattle (1993),Comedy|Romance
921,1036,Die Hard (1988),Action|Thriller
2017,2302,My Cousin Vinny (1992),Comedy
1479,1682,"Truman Show, The (1998)",Drama
1088,1219,Psycho (1960),Horror|Thriller
1444,1645,"Devil's Advocate, The (1997)",Crime|Horror|Mystery|Thriller
2885,3247,Sister Act (1992),Comedy|Crime
1498,1704,Good Will Hunting (1997),Drama
1428,1625,"Game, The (1997)",Mystery|Thriller
2754,3107,Backdraft (1991),Action|Drama


In [ ]:
# Top 20 movies that User 4375 has rated
already_rated.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
195,4375,3250,5,4374,3249,Alive (1993),Drama
285,4375,175,5,4374,174,Kids (1995),Drama
199,4375,3257,5,4374,3256,"Bodyguard, The (1992)",Action|Drama|Romance|Thriller
65,4375,3809,5,4374,3808,What About Bob? (1991),Comedy
210,4375,2688,5,4374,2687,"General's Daughter, The (1999)",Drama|Thriller
270,4375,150,5,4374,149,Apollo 13 (1995),Drama
271,4375,2710,5,4374,2709,"Blair Witch Project, The (1999)",Horror
125,4375,349,5,4374,348,Clear and Present Danger (1994),Action|Adventure|Thriller
60,4375,288,5,4374,287,Natural Born Killers (1994),Action|Thriller
141,4375,47,5,4374,46,Seven (Se7en) (1995),Crime|Thriller


#### Evaluation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data, test_data = train_test_split(ratings, test_size=0.2)

train_ratings = train_data.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
R_train = train_ratings.values


U_train, sigma_train, Vt_train = svds(R_train, k = 50)
sigma_train = np.diag(sigma_train)

all_user_predicted_ratings_train = np.dot(np.dot(U_train, sigma_train), Vt_train)
preds_train = pd.DataFrame(all_user_predicted_ratings_train, columns = train_ratings.columns)

def predict_rating(user_id, movie_id):
    user_row_number = user_id - 1
    movie_col_number = movie_id - 1
    if user_row_number >= len(preds_train) or movie_col_number >= len(preds_train.columns):
        return np.nan
    pred = preds_train.iloc[user_row_number, movie_col_number]
    return pred

test_data['predicted_rating'] = test_data.apply(lambda x: predict_rating(x['user_id'], x['movie_id']), axis=1)

test_data.dropna(inplace=True)

rmse = sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 3.5060265025430923


### Nueral Network Model (Recommender Model)



**Objective**: Implement a Recommender model to recommend movies to a user based on similar users' preferences.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Define Dataset and DataLoader**:
   - Create a custom PyTorch `Dataset` for ratings.
   - Create a DataLoader for batching and shuffling data.

2. **Define the Neural Network**:
   - Create a neural network with embedding layers for users and movies.

3. **Train the Model**:
   - Train the model using Mean Squared Error loss and Adam optimizer.
   - Save model checkpoints.

4. **Evaluate the Model**:
   - Calculate RMSE on the entire dataset.

5. **Predict Ratings for Unrated Movies**:
   - Predict and recommend top 10 unrated movies for a given user.

In [3]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from tqdm.notebook import tqdm


In [22]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class RatingsDataset(Dataset):
    def __init__(self, ratings):
        self.user_ids = torch.tensor(ratings['user_id'].values, dtype=torch.long).to(device)
        self.movie_ids = torch.tensor(ratings['movie_id'].values, dtype=torch.long).to(device)
        self.ratings = torch.tensor(ratings['rating'].values, dtype=torch.float32).to(device)



    def __len__(self):
        return len(self.ratings)


    def __getitem__(self, idx):
        return self.user_ids[idx], self.movie_ids[idx], self.ratings[idx]

class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=50):
        super(RecommenderNet, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        self.fc1 = nn.Linear(embedding_size*2, 128)
        self.ac1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.ac2 = nn.Sigmoid()
        self.fc3 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.2)


    def forward(self, user, movie):
        user_embedding = self.user_embedding(user)
        movie_embedding = self.movie_embedding(movie)
        x = torch.cat([user_embedding, movie_embedding], dim=1)
        x = self.fc1(x)
        x = self.ac1(x)
        x = self.fc2(x)
        x = self.ac2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x
print(device,ratings.shape)
ratings_dataset = RatingsDataset(ratings)
ratings_dataloader = DataLoader(ratings_dataset, batch_size=512, shuffle=True)

num_users = ratings['user_id'].max()
num_movies = ratings['movie_id'].max()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RecommenderNet(num_users, num_movies, embedding_size=50).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

n_epochs = 10
for epoch in tqdm(range(n_epochs)):
    model.train()
    total_loss = 0
    for i, (user, movie, rating) in enumerate(ratings_dataloader):
        optimizer.zero_grad()
        output = model(user - 1, movie - 1)
        loss = criterion(output, rating.view(-1, 1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch + 1}/{n_epochs}, Loss: {total_loss / len(ratings_dataloader):.4f}')

torch.save(model.state_dict(), "nn")

# Evaluation
model.eval()
predictions = []
with torch.no_grad():
    for i, (user, movie, rating) in enumerate(ratings_dataloader):
        user, movie = user.to(device), movie.to(device)
        output = model(user - 1, movie - 1)
        predictions.extend(output.view(-1).tolist())


cuda (1000209, 5)


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10, Loss: 1.1239
Epoch 2/10, Loss: 0.8787
Epoch 3/10, Loss: 0.8473
Epoch 4/10, Loss: 0.8295
Epoch 5/10, Loss: 0.8157
Epoch 6/10, Loss: 0.8050
Epoch 7/10, Loss: 0.7961
Epoch 8/10, Loss: 0.7881
Epoch 9/10, Loss: 0.7809
Epoch 10/10, Loss: 0.7740


In [18]:
sample_user_id = 1
sample_user_data = ratings[ratings['user_id'] == sample_user_id]
sample_dataset = RatingsDataset(sample_user_data)
sample_dataloader = DataLoader(sample_dataset, batch_size=1, shuffle=False)

print(f'Sample predictions for user ID {sample_user_id}:')
with torch.no_grad():
    for user, movie, rating in sample_dataloader:
        output = model(user, movie).squeeze()
        print(f'Movie ID: {movie.item()}, Predicted Rating: {output.item()}, Actual Rating: {rating.item()}')

Sample predictions for user ID 1:
Movie ID: 1193, Predicted Rating: 3.47823429107666, Actual Rating: 5.0
Movie ID: 661, Predicted Rating: 2.4390077590942383, Actual Rating: 3.0
Movie ID: 914, Predicted Rating: 3.8591501712799072, Actual Rating: 3.0
Movie ID: 3408, Predicted Rating: 3.320901393890381, Actual Rating: 4.0
Movie ID: 2355, Predicted Rating: 2.7952260971069336, Actual Rating: 5.0
Movie ID: 1197, Predicted Rating: 4.447009086608887, Actual Rating: 3.0
Movie ID: 1287, Predicted Rating: 4.103301525115967, Actual Rating: 5.0
Movie ID: 2804, Predicted Rating: 2.8952901363372803, Actual Rating: 5.0
Movie ID: 594, Predicted Rating: 3.8719916343688965, Actual Rating: 4.0
Movie ID: 919, Predicted Rating: 4.061120986938477, Actual Rating: 4.0
Movie ID: 595, Predicted Rating: 3.716154098510742, Actual Rating: 5.0
Movie ID: 938, Predicted Rating: 3.0957388877868652, Actual Rating: 4.0
Movie ID: 2398, Predicted Rating: 2.4190704822540283, Actual Rating: 4.0
Movie ID: 2918, Predicted Rati

In [7]:
model.to('cpu')
def predict_unrated_movies(user_id, model, ratings, movies):
    user_data = ratings[ratings['user_id'] == user_id]
    user_movies = set(user_data['movie_id'])
    unrated_movies = movies[~movies['movie_id'].isin(user_movies)]
    unrated_movie_ids = unrated_movies['movie_id'].values
    user_ids = torch.tensor([user_id]*len(unrated_movie_ids), dtype=torch.long)
    movie_ids = torch.tensor(unrated_movie_ids, dtype=torch.long)
    with torch.no_grad():
        predictions = model(user_ids-1, movie_ids-1).squeeze().tolist()
    predictions = pd.DataFrame({'movie_id': unrated_movie_ids, 'predicted_rating': predictions})
    predictions = pd.merge(movies, predictions, on='movie_id')
    return predictions.sort_values('predicted_rating', ascending=False)





# Example
user_id = 1
predictions = predict_unrated_movies(user_id, model, ratings, movies)
print(predictions)

      movie_id                                             title  \
3248      3369  Any Number Can Win (Mélodie en sous-sol ) (1963)   
2912      3030                                    Yojimbo (1961)   
2618      2731    400 Blows, The (Les Quatre cents coups) (1959)   
894        922     Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)   
2392      2503                           Apple, The (Sib) (1998)   
...        ...                                               ...   
3147      3268                 Stop! Or My Mom Will Shoot (1992)   
2700      2817                       Aces: Iron Eagle III (1992)   
1435      1495               Turbo: A Power Rangers Movie (1997)   
786        810                                     Kazaam (1996)   
3471      3593                          Battlefield Earth (2000)   

                           genres  predicted_rating  
3248                        Crime          5.123587  
2912         Comedy|Drama|Western          5.012771  
2618                 

### GMM (Gaussian Mixture Model)

**Objective**: Use a Gaussian Mixture Model to analyze and cluster the click data based on the number of clicks from different locations, aiming to identify distinct patterns of user behavior across 10 countries.

**Dataset**:
- **Click Data**: DataFrame with columns `link_id`, `location`, and `number_of_clicks`.

**Steps**:

1. **Preprocess Data**:
   - Ensure the dataset contains 10 distinct countries.
   - Create a matrix where rows represent different links and columns represent the number of clicks from each country.
   - Normalize the number of clicks to account for different scales.

2. **Fit GMM**:
   - Apply a Gaussian Mixture Model (GMM) to the click data matrix to identify clusters of links with similar click patterns across different countries.

3. **Evaluate Model**:
   - Analyze the resulting model by calculating the log lokelihood, BIC and AIC metrics.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

file_path = "/content/drive/MyDrive/gmm-dataset.csv"
df = pd.read_csv(file_path)
df = df.iloc[:, 1:]
mean = df.mean()
std = df.std()
df = (df - mean) / std

gmm = GaussianMixture(n_components=11)
gmm.fit(df)
clusters = gmm.predict(df)
df['Cluster'] = clusters
print(df)


log_likelihood = gmm.score(df.iloc[:, :-1])
bic = gmm.bic(df.iloc[:, :-1])
aic = gmm.aic(df.iloc[:, :-1])

print(f"Log-Likelihood: {log_likelihood}")
print(f"BIC: {bic}")
print(f"AIC: {aic}")

    United States     China     India     Japan    Brazil    Russia  \
0        0.549942 -1.150906 -0.731604  0.891758  0.901695 -0.116030   
1       -0.392775 -1.312975 -0.578585  0.681981  0.414217  1.677038   
2        1.682610 -0.138875 -0.039541  0.819540 -0.844811  0.846715   
3        0.082101  0.667868  0.596878  0.561617 -0.132882 -1.103829   
4        0.936878  0.531010 -0.700305  0.142064 -0.392403 -0.241295   
..            ...       ...       ...       ...       ...       ...   
95       0.064513  0.621048  1.094189 -1.608370  0.161710 -1.204040   
96      -1.208859  0.833539 -0.912445 -0.559485  0.670231 -1.139618   
97       1.686128  0.088021 -0.265592  0.434376 -1.398923  0.023550   
98       0.430343  0.657064  0.600356  0.062967  0.063513  0.381448   
99       0.514766  1.694305  0.659477 -0.397854  0.691273  1.723565   

    Indonesia   Germany  United Kingdom    France  Cluster  
0   -0.971418  1.688887       -0.056640 -0.656355        2  
1   -0.455322  1.668934  